<a href="https://colab.research.google.com/github/ConnBreathnach/LiloGuesser/blob/main/Chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test_games.csv to test_games.csv
Saving train_games.csv to train_games.csv


In [2]:

VOCAB_SIZE = 10863
BATCH_SIZE=1024
data = pd.read_csv("train_games.csv")
data.dropna(inplace=True)

data['white_min_elo'] = pd.Categorical(data['white_min_elo'])
data['white_min_elo'] = data.white_min_elo.cat.codes

features = data.pop('moves').astype("string")
labels = data.pop('white_min_elo')

dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(BATCH_SIZE)


test_data = pd.read_csv("test_games.csv")
test_data.dropna(inplace=True)
test_data['white_min_elo'] = pd.Categorical(test_data['white_min_elo'])
test_data['white_min_elo'] = test_data.white_min_elo.cat.codes
test_features = test_data.pop('moves').astype("string")
test_labels = test_data.pop('white_min_elo')
test_dataset = tf.data.Dataset.from_tensor_slices((test_features,test_labels)).batch(BATCH_SIZE)


FileNotFoundError: ignored

In [ ]:
vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=150
)

In [ ]:
vectorize_layer.adapt(dataset.map(lambda text, labels: text))

In [ ]:
def get_model():
    model = tf.keras.Sequential(
        [
            vectorize_layer,
            tf.keras.layers.Embedding(
                input_dim=len(vectorize_layer.get_vocabulary()),
                output_dim=512,
                mask_zero=True
            ),
            tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(128, activation='tanh'),
            tf.keras.layers.Dense(21)
        ]
    )
    model.compile(optimizer="SGD", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model


In [ ]:

model = get_model()
sample_game = "e4 e5 bc4 nc6 qf3 a6 qxf7#"
model.predict(np.array([sample_game]))
history = model.fit(dataset, epochs=5, validation_data=test_dataset, validation_steps=15)